<a href="https://colab.research.google.com/github/Hellavn/notebooks/blob/main/BreastCancer_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Deep Learning Project

##Dataset: [ Breast Cancer Dataset](https://www.kaggle.com/datasets/yasserh/breast-cancer-dataset/code)

###ETL

In [143]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix


In [103]:
#!kaggle datasets download -d andrewmvd/medical-mnist

In [104]:
df = pd.read_csv('/content/breastcancerdataset/breast-cancer.csv')

In [105]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [106]:
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [107]:
df['diagnosis_maligno'] = df['diagnosis'].apply(lambda x: 1 if x=='M' else 0)

In [108]:
df['diagnosis_maligno'].value_counts()

0    357
1    212
Name: diagnosis_maligno, dtype: int64

In [109]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'diagnosis_maligno'],
      dtype='object')

In [110]:
x = df[['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']]


y = df['diagnosis_maligno']

In [111]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
x_train, x_validation, y_train, y_validation = train_test_split(x_train , y_train, test_size = 0.1)

In [112]:
len(x_train)

460

In [113]:
len(x_test)

57

In [114]:
len(x_validation)

52

In [115]:
sc = StandardScaler()

In [116]:
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_validation = sc.transform(x_validation)

In [117]:
x_train

array([[-1.11147033, -1.67295106, -1.09473134, ..., -0.71591847,
         0.06455647, -0.3455621 ],
       [ 1.82936261,  0.57716037,  1.87867038, ...,  2.37913912,
         4.30475904,  1.00790807],
       [ 0.30059628, -0.6629441 ,  0.33919853, ...,  0.44108083,
        -0.49681334, -0.31492358],
       ...,
       [-0.07284282,  1.29893993,  0.03189756, ...,  0.59157981,
         1.21965172,  1.86256159],
       [ 0.32685372, -1.43316523,  0.25739289, ..., -0.48832716,
        -0.66344473, -0.85942925],
       [ 0.49023332,  0.02250427,  0.6579438 , ...,  1.53450199,
         2.18061331,  1.14766273]])

In [118]:
x_train.shape

(460, 30)

###Definition of architecture

In [135]:
def build_model():
    model = Sequential()
    model.add(Dense(input_dim=30, units=10, activation="relu"))
    model.add(Dense(10, activation="relu"))

    model.add(Dense(1, activation="sigmoid"))

    return model

In [136]:
model = build_model()


In [137]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 10)                310       
                                                                 
 dense_19 (Dense)            (None, 10)                110       
                                                                 
 dense_20 (Dense)            (None, 1)                 11        
                                                                 
Total params: 431
Trainable params: 431
Non-trainable params: 0
_________________________________________________________________


###Compilation

In [138]:
adam = Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

###Training

In [140]:
model.fit(x=x_train, y=y_train, validation_data=(x_validation,y_validation), batch_size=16, epochs=10)

Epoch 1/10
29/29 [==============================] - 0s 5ms/step - loss: 0.0200 - accuracy: 0.9935 - val_loss: 0.2582 - val_accuracy: 0.9423
Epoch 2/10
29/29 [==============================] - 0s 4ms/step - loss: 0.0155 - accuracy: 0.9935 - val_loss: 0.2637 - val_accuracy: 0.9231
Epoch 3/10
29/29 [==============================] - 0s 3ms/step - loss: 0.0183 - accuracy: 0.9935 - val_loss: 0.2493 - val_accuracy: 0.9423
Epoch 4/10
29/29 [==============================] - 0s 5ms/step - loss: 0.0135 - accuracy: 0.9935 - val_loss: 0.2618 - val_accuracy: 0.9231
Epoch 5/10
29/29 [==============================] - 0s 5ms/step - loss: 0.0108 - accuracy: 0.9957 - val_loss: 0.3746 - val_accuracy: 0.9231
Epoch 6/10
29/29 [==============================] - 0s 5ms/step - loss: 0.0281 - accuracy: 0.9891 - val_loss: 0.3098 - val_accuracy: 0.9038
Epoch 7/10
29/29 [==============================] - 0s 5ms/step - loss: 0.0539 - accuracy: 0.9783 - val_loss: 0.3172 - val_accuracy: 0.9423
Epoch 8/10
29/29 [==

###Evaluation

In [144]:
y_pred = model.predict(x_test)

2/2 [==============================] - 0s 4ms/step


In [146]:
cm = confusion_matrix(y_test, y_pred>0.5)

In [147]:
print(cm)

[[33  1]
 [ 0 23]]
